## Load libraries

In [ ]:
import json
import os
import urllib

import requests
import polling


## Set token/URL

The Python `requests` library expects a single URL, rather than a URL plus a path, hence we define a URL that includes the path to the simulation calls.

In [ ]:
token = os.getenv("API_TOKEN")
assert token

base_path = os.getenv("API_BASE_URL", "https://covid-modelling-stg.epcc.ed.ac.uk")

In [ ]:
model_slug = "sir-ode-python"
headers = {
    "accept":"application/json; charset=utf-8", 
    "Content-Type": "application/json", 
    "Authorization": "Bearer "+ token
}

## Input

In [ ]:
inputJSON = {
  "p": [
    0.5,
    0.25
  ],
  "u0": [
    0.99,
    0.01,
    0
  ],
  "tspan": [
    0,
    10000
  ]
}

## Post simulations

In [ ]:
post_simulations_url = "/".join([base_path, "api", "simulations", "model-runs", model_slug])
post_simulations_url

In [ ]:
post_simulations_response = requests.post(post_simulations_url, json = inputJSON, headers = headers)

assert post_simulations_response.status_code == 200

post_simulations_content = post_simulations_response.json()
post_simulations_content

We need the ID to construct other URLs later.

In [ ]:
simulation_id = str(post_simulations_content["id"])

## Check simulations

In [ ]:
check_simulations_url = "/".join([base_path, "api", "simulations", simulation_id, "model-runs", model_slug])
check_simulations_url

In [ ]:
check_simulations_response = requests.get(check_simulations_url, headers = headers)

assert check_simulations_response.status_code == 200

check_simulations_content = check_simulations_response.json()
check_simulations_content

The process can take some time to complete.
The following code polls the API to check for success every `step` seconds, with a `timeout` in seconds.

In [ ]:
step = 60
timeout = 600

def is_complete(response):
    """Check that the response returned 'complete'"""
    return response.json()["status"] == "complete"

try:
    check_simulations_response = polling.poll(
        lambda: requests.get(check_simulations_url, headers = headers),
        check_success=is_complete,
        step=step,
        timeout=timeout)
except polling.TimeoutException:
    print(check_simulations_url+" failed")

In [ ]:
check_simulations_content = check_simulations_response.json()
check_simulations_content

# Export simulation results

In [ ]:
export_simulations_url = "/".join([base_path, "api", "simulations", simulation_id, "model-runs", model_slug, "export"])
export_simulations_url

In [ ]:
export_simulations_response = requests.get(export_simulations_url, headers = headers)

assert export_simulations_response.status_code == 200

export_simulations_content = export_simulations_response.json()
export_simulations_content

# Download simulation output

If we need access to the full output and logs of the simulation, for example for debugging purposes, we can get that via the `download` endpoint.

In [ ]:
download_simulations_url = "/".join([base_path, "api", "simulations", simulation_id, "model-runs", model_slug, "download"])
download_simulations_url

This endpoint returns a `307 Redirect` to another URL where the contents can be accessed (as a `.zip`). We could follow this and save them to a file in code, but for now we're just going to print the URL.

In [ ]:
download_simulations_response = requests.get(download_simulations_url, headers = headers, allow_redirects=False)

assert download_simulations_response.status_code == 307

download_simulations_response.headers['Location']

Copy and paste the URL above into a browser, and you'll be able to download the files to your machine.

# Groups of simulations

The API can also be used to run multiple models on the same set of inputs, in the same way as the UI itself. This uses a different input schema.

In [ ]:
inputJSON = {
  "regionID": "GB",
  "subregionID": "GB-ENG",
  "label": "Simulation",
  "customCalibrationDate": "2020-03-06",
  "interventionPeriods": [
    {
      "startDate": "2020-03-06",
      "socialDistancing": "aggressive",
      "schoolClosure": "aggressive",
      "caseIsolation": "aggressive",
      "voluntaryHomeQuarantine": "aggressive",
      "reductionPopulationContact": 0
    }
  ]
}

In [ ]:
post_simulations_url = "/".join([base_path, "api", "simulations"])
post_simulations_url

In [ ]:
post_simulations_response = requests.post(post_simulations_url, json = inputJSON, headers = headers)

assert post_simulations_response.status_code == 200

post_simulations_content = post_simulations_response.json()
post_simulations_content

simulation_id = str(post_simulations_content["id"])

This time, we want to check the status of the top-level simulation

In [ ]:
check_simulations_url = "/".join([base_path, "api", "simulations", simulation_id])
check_simulations_url

In [ ]:
check_simulations_response = requests.get(check_simulations_url, headers = headers)

assert check_simulations_response.status_code == 200

check_simulations_content = check_simulations_response.json()
check_simulations_content

Again, we need to poll until the simulations are complete.

In [ ]:
step = 60
timeout = 600

def is_complete(response):
    """Check that the response returned 'complete'"""
    for model in response.json()["model_runs"]:
        if model["status"] in ["pending", "in-progress"]:
            return False
    return True

try:
    check_simulations_response = polling.poll(
        lambda: requests.get(check_simulations_url, headers = headers),
        check_success=is_complete,
        step=step,
        timeout=timeout)
except polling.TimeoutException:
    print(check_simulations_url+" failed")

In [ ]:
check_simulations_content = check_simulations_response.json()
check_simulations_content

We can then retrieve a summary of the results across the models:

In [ ]:
case_summary_url = "/".join([base_path, "api", "simulations", simulation_id, "case-summary"])
case_summary_url

In [ ]:
case_summary_response = requests.get(case_summary_url, headers = headers)

assert case_summary_response.status_code == 200

case_summary_content = case_summary_response.json()
case_summary_content

If we want to get the details of a particular model, we can use the same endpoint as before.

In [ ]:
export_simulations_url = "/".join([base_path, "api", "simulations", simulation_id, "model-runs", "wss", "export"])
export_simulations_url

In [ ]:
export_simulations_response = requests.get(export_simulations_url, headers = headers)

assert export_simulations_response.status_code == 200

export_simulations_content = export_simulations_response.json()
export_simulations_content

Some models also support a `format` parameter, in order to get results back in alternatives formats.

In [ ]:
export_simulations_response = requests.get(export_simulations_url, params = { "format": "crystalcast" }, headers = headers)

assert export_simulations_response.status_code == 200

export_simulations_content = export_simulations_response.text
export_simulations_content

# Errors

The following submits a simulation with the wrong configuration for the chosen model.

In [ ]:
post_simulations_url = "/".join([base_path, "api", "simulations", "model-runs", "wss"])
post_simulations_url

In [ ]:
post_simulations_response = requests.post(post_simulations_url, json = inputJSON, headers = headers)

This time, we expect to get back a `422` response

In [ ]:
assert post_simulations_response.status_code == 422

And we can see the error description:

In [ ]:
post_simulations_response.json()